In [1]:
#Purpose: Do change point fitting to time series data with fixed time-binwidths and gaussian noise
#Adapted From: Li and Yang, J Phys Chem B, 123, 689-701 (2019) https://pubs.acs.org/doi/full/10.1021/acs.jpcb.8b10561
#Written By: Jessica Kline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Imports
import matplotlib.pyplot as plt
import numpy as np

from random import seed
from random import random
from random import shuffle
from scipy.stats import norm
from scipy.stats import rankdata
from scipy.optimize import curve_fit
from scipy.special import gammainc
from scipy.special import gamma
import copy
import bisect
import math
import mpmath
import pandas as pd
import os

from numba import jit

!pip install bottleneck
import bottleneck as bn

! pip install line_profiler
%load_ext line_profiler

from scipy.signal import argrelextrema

import matplotlib.patches as patches

import warnings
warnings.filterwarnings("ignore")

seed(1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 14.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.6/673.6 KB 23.9 MB/s eta 0:00:00


In [4]:
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 22
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['xtick.major.pad']='10'
plt.rcParams['ytick.major.pad']='4'

In [5]:
my_colors = ['dimgrey', [242/255, 133/255, 125/255], [254/255, 185/255, 95/255], [126/255, 188/255, 118/255],'tab:blue']

In [6]:
#@title create_data(int delta_I)
# this code creates synthetic data of a variable number of intensity levels following a 1.5 powerlaw
# the intensity levels range from max_I to (max_I - n*delta_I > min_I)
def create_data(max_I, min_I, delta_I, delta_t):
  #generate list of delta-time changepoints assuming power law exponent of 1.5
  time = [float(x*delta_t) for x in range(1,2001)]
  P_t = [t**-1.5 for t in time]
  P_selec = []
  t_selec = []
  for n in range (80):
    P_selec.append(len(time)*random())
  P_selec = np.sort(P_selec)
  t_selec = P_selec
  change_times = [time[int(t_selec[n])] for n in range(len(t_selec))]

  #create a list of the number of events associated with each change point assuming a power law with exponent of 1.5
  events = []
  for n in range(len(P_selec)):
    if n == 0:
      a = 0
    else:
      a = time[int(t_selec[n-1])]
    if n == len(P_selec)-1:
      b = time[-1]
    else:
      b = time[int(t_selec[n+1])]
    events.append(time[int(t_selec[n])]**-1.5 *(a+b)/2)

  #generate a list of randomly ordered real-time changpoints
  events = events/np.amin(events)
  total_time = np.sum(np.multiply(events, change_times))
  f = time[-1]
  time_list = []
  for n in range(len(events)):
    cnt = 0
    while cnt < events[n]:
      time_list.append(change_times[n])
      cnt = cnt + 1
  shuffle(time_list)

  event_dist_num = random()
  
  #create list of intensities based on the changepoints
  I = []
  act_cp = []
  current_I = max_I             #change this number to change the maximum simulated intensity
  x_t = 0
  mult = 1
  for n in range(len(time_list)):
    cnt = 0
    while cnt < time_list[n]:
      I.append(np.random.normal(current_I, 0.03*current_I))
      cnt = cnt + delta_t
      x_t = x_t + delta_t
    if current_I == max_I or current_I == min_I:
      mult = 1
    else:
      mult = 3
    if mult*random() >= event_dist_num:
      current_I = current_I-delta_I
    if current_I < min_I:     #change this number to change the minimum simulated intensity
      current_I = max_I       #change this number to change the maximum simulated intensity
    if (x_t-delta_t < 400):
      act_cp.append(x_t-delta_t)

  #trim intensity and time lists to about 7 mins
  time = [float(x*delta_t) for x in range(0,8000)]
  I = I[0:8000]
  time = time[0:8000]


  data = I
  data = np.array(data)
  return data, time, act_cp

In [7]:
#@title helper functions

def linear_power_law(t, C, alpha):
  #returns a linearized power law y = Ct^(-alpha)
  return -alpha*np.log(t) + np.log(C)

def linear_trunc_power_law(t, C, alpha, T_c):
  #returns a linearized power law with truncation factor y = Ct^(-alpha)e^(t/Tc)
  return -alpha*np.log(t) + np.log(C) + t/T_c

def R_sq(fit, ydata):
  #cacluates the r^2 of a fit
  residual = ydata-fit
  ss_resid = np.sum(residual**2)
  ss_tot = np.sum((ydata-np.mean(ydata))**2)
  return 1- (ss_resid/ss_tot)

@jit
def gauss(x, I, sigma):
  #returns the value of a gaussian with central mean I and std_dev sigma for value x
  return np.exp(-1*(((x-I)/sigma)**2)/2)/(sigma*math.sqrt(2*math.pi))

def CriVal(N,alpha):
  # calculate the critical value (cv) given the number of data point(N) and type-I
  # error rate. 

  yd=-math.log(-1/2*math.log(alpha));
  x=math.log(N);
  a=math.sqrt(2*math.log(x));
  b=2*math.log(x)+math.log(math.log(x));
  cv=(yd+b)/a;

  return cv

In [8]:
#@title ndarray cp = findcp(ndarray data)
#find the change points in the data

def findcp(data):
  #empty array to contain all the data
  traj = [[]]*1

  #first array box is the entire set of data
  traj[0] = list(data)
  
  n = 0
  while n < (len(traj)):    #iterate over the length of the entire segmented trace
    f = [[1]]*2
    while len(traj[n]) > 4 and bool(list(f[0])):    #iterate over each segment to find all change points
      L = np.zeros([len(data),1])
      cri_val = np.zeros([len(data),1])      
      sigma_0 = bn.nanstd(traj[n])

      # caculate the Log_likelihood of a change point (L) and the critical value (95% confidence) which it must be above for a change point to have occured
      for k in range(3, len(traj[n])-2):
        sigma_1 = bn.nanstd(traj[n][0:k+1])
        sigma_2 = bn.nanstd(traj[n][k+1:len(traj[n])])

        if sigma_2 == 0:
          sigma_2 = 1e-12
        if sigma_1 == 0:
          sigma_1 = 1e-12
        if sigma_0 == 0:
          sigma_0 = 1e-12
        L[k] = -(k)/2*math.log(sigma_1**2) - (len(traj[n])-k)/2*math.log(sigma_2**2) + (len(traj[n]))/2*math.log(sigma_0**2)
        cri_val[k] = CriVal(k, 1-0.2)
      ## end for

      #find the point R greater than the critical where it is most likely for a change point to have occured
      Z = np.sqrt(2*L)
      z_greater = np.greater(Z, cri_val)
      Z = np.multiply(Z, z_greater) 
      R = np.argmax(Z)
      
      #split data around change point R and adjust trajectory array accordingly
      f = np.split(traj[n],[R])
      if len(f[0]) > 0:
        traj[n] = f[0]
        traj.insert(n+1,f[1])
    ## end while 2
    n = n+1
  ## end while 1

  #convert delta_points into the real_point locations of the change points
  prev_cp = 0;
  all_cp = []
  for n in range(len(traj)):
    all_cp.append(len(traj[n])+prev_cp)
    prev_cp = prev_cp + len(traj[n])

  all_cp = np.unique(all_cp)

  return all_cp

In [9]:
#@title dict Yi = AHclusterN(ndarray traj,  ndarray cp)
#This section does agglomerate hierarcy clustering. Essentially what is does is
#   using the changepoints calculated previously determine the median intensity and the std_dev of each section
#   and then do a log-likelyhood test to see which two sections should be combined until the data is all assigned to one intensity level
#   all n of these groupings are returned

def concat(tr_1, tr_2):
  tr_concat = np.zeros([len(tr_1)+len(tr_2)])
  tr_concat[0:len(tr_1)] = tr_1
  tr_concat[len(tr_1):len(tr_1)+len(tr_2)] = tr_2
  return tr_concat

# calculate the log-likelyhood
def MN_Calc(section_1, section_2):
  temp = concat(section_1['tr'], section_2['tr'])
  comb_sigma = bn.nanstd(temp)
  if comb_sigma == 0:
    comb_sigma = 1e-12
  comb_I = bn.median(temp)
  return math.log(section_1['std']*section_2['std']/(comb_sigma**2)) - ((section_1['intensity'] - comb_I)**2 + (section_2['intensity'] - comb_I)**2)/(2*(comb_sigma**2))

#turn array of different sized lists into a square matrix with empty cells filled by NaN
def boolean_indexing(v):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max()+1)
    mask = np.fliplr(mask)
    out = np.zeros(mask.shape,dtype=float)
    out[:] = np.NaN
    out[mask] = np.concatenate(v)
    return out

#AH clustering process
def AHclusterN(traj,cp):
  cp.append(len(traj))
  Ng_max = len(cp)
  Yi = [{'tr':0, 'intensity':0, 't':0, 'group':[0 for x in range(Ng_max)], 'std':0} for i in range(Ng_max)]

  # determine inital len(cp) sections intensities and std_devs
  for i in range(Ng_max):
    if i == 0:
      start_ = 0
      end_ = cp[i]+1
    else:
      start_ = cp[i-1]+1
      end_ = cp[i]+1
    Yi[i]['tr'] = traj[start_:end_]
    Yi[i]['intensity'] = bn.median(Yi[i]['tr'])
    Yi[i]['std'] = bn.nanstd(Yi[i]['tr'])
    if Yi[i]['std'] == 0:
      Yi[i]['std'] = 1E-50
    Yi[i]['group'][-1] = i
    Yi[i]['t'] = len(Yi[i]['tr'])

  Yi_comb = copy.deepcopy(Yi)

  M_mn = [[MN_Calc(Yi_comb[m], Yi_comb[n]) for m in range(n+1, len(Yi_comb))] for n in range(len(Yi_comb))]
  M_mn = boolean_indexing(M_mn).T


  # combine states to have len(cp)-1:1 states
  for i in range(len(Yi) - 1, 0, -1):
    mask = np.ones(M_mn.shape, dtype=bool)
    loc_mn= np.unravel_index(bn.nanargmax(M_mn, axis=None), M_mn.shape)
    mn_max = np.amax(loc_mn)
    mn_min = np.amin(loc_mn)

    group_min = np.amin([Yi_comb[mn_max]['group'][i],Yi_comb[mn_min]['group'][i]])
    group_max = np.amax([Yi_comb[mn_max]['group'][i],Yi_comb[mn_min]['group'][i]])

    # change state assignments for the selected states to combine
    for nn in range(len(Yi)):
      if Yi[nn]['group'][i] == group_min or Yi[nn]['group'][i] == group_max:
        Yi[nn]['group'][i-1] = group_min
      else:
        Yi[nn]['group'][i-1] = Yi[nn]['group'][i]

    for nn in range(len(Yi_comb)):
      Yi_comb[nn]['group'][i-1] = Yi_comb[nn]['group'][i]

    # combine the two states
    Yi_comb[mn_min]['tr'] = np.concatenate((Yi_comb[mn_max]['tr'], Yi_comb[mn_min]['tr']))
    Yi_comb[mn_min]['intensity'] = bn.median(Yi_comb[mn_min]['tr'])
    Yi_comb[mn_min]['std'] = bn.nanstd(Yi_comb[mn_min]['tr'])
    if Yi_comb[mn_min]['std'] == 0:
      Yi_comb[mn_min]['std'] = 1E-50
    Yi_comb[mn_min]['group'][i-1] = group_min
    Yi_comb.pop(mn_max)


    #delete set of rows/columns associated with segment that got comibined
    mask[:][mn_max] = False
    mask = mask.T
    mask[:][mn_max] = False
    M_mn = np.reshape(M_mn[mask], [len(M_mn)-1, len(M_mn)-1])

    #recalculate values in M_mn affected by the combining of the two segments
    M_mn[mn_min][:] = [MN_Calc(Yi_comb[mn_min], Yi_comb[n]) if mn_min > n else math.nan for n in range(len(M_mn[0]))]

    M_mn[:][mn_min] = [MN_Calc(Yi_comb[m], Yi_comb[mn_min])  if mn_min < m else math.nan for m in range(len(M_mn[0]))]


        
  # change the state assignment numbers such that they range [0, n] w/ step = 1
  for n in range(len(Yi[0]['group'])):
    group_lst = [Yi[m]['group'][n] for m in range(len(Yi))]
    group_rank = rankdata(group_lst, method = 'dense')
    for m in range(len(Yi)):
      Yi[m]['group'][n] = group_rank[m]-1
  
  return Yi

In [10]:
#@title dict Yem = EMclusterN(dict Yi)
# This takes the data from AHclusterN and decides which intensity levels are actually the same
#   it runs on every returned grouping from AHclusterN and can only decrease the number of intensity levels

def EMclusterN(Yi):
  Yem = [[{'state':0, 'intensity':0, 'sigma':0, 'prob':0, 'pk':0, 'nos':0} for i in range(len(Yi))] for j in range(len(Yi))]
  array_of_states = np.zeros([len(Yi), len(Yi)])
  array_of_intensities = np.zeros([len(Yi), len(Yi)])
  array_of_lengths = np.zeros([len(Yi), len(Yi)])
  array_of_stddevs_sq = np.zeros([len(Yi), len(Yi)])


  # create stock arrays of the various intensities, lengths and std_devs for all the segments
  for r in range(len(Yi)):
    for j in range(len(Yi)):
      array_of_states[j, r] = Yi[r]['group'][j]
      array_of_intensities[j,r] = Yi[r]['intensity']
      array_of_lengths[j,r] = Yi[r]['t']
      array_of_stddevs_sq[j,r] = Yi[r]['std']**2



  for total_clusters in range(len(Yi)):

    #for runtime purposes complete maximization is only run on the AH results from 10 possible states on down
    if total_clusters < 10:

      #STEP 1
      #p_mj is the array which represents the probability that state m belongs to state j
      #it's initialized with the AH results

      num_states = np.unique(np.transpose(array_of_states[total_clusters][:]))
      j_length = len(Yi[0]['group'])
      m_length = int(np.amax(num_states)+1)
      delta_p_mj = 1

      intensity_arr = array_of_intensities[:][0:m_length]
      len_arr = array_of_lengths[:][0:m_length]
      stddev_arr = array_of_stddevs_sq[:][0:m_length]

      p_mj = np.zeros([len(Yi[0]['group']), int(np.amax(num_states)+1)])
      for n in range(len(Yi[0]['group'])):
        p_mj[n][Yi[n]['group'][total_clusters]] = 1
      p_mj = np.transpose(p_mj)

      iter = 0
      #this while loop runs until there is a sufficently small change in p_mj or the maximum number of iterations is exceeded
      while delta_p_mj > 1e-8 and iter < 1e4:

        #STEP 2
        old_p_mj = np.copy(p_mj)

        #calculate the intensity (I_m), std_dev (sigma_m) and probabilties (P_m) of every state
        p_mj_x_N_j = np.multiply(p_mj, len_arr)
        I_m = bn.nansum(np.multiply(intensity_arr, p_mj_x_N_j), axis = 1)/bn.nansum(p_mj_x_N_j, axis = 1)
        sigma_m = np.sqrt(np.sum(np.multiply(stddev_arr, p_mj_x_N_j), axis = 1)/bn.nansum(p_mj_x_N_j, axis = 1))
        P_m = bn.nansum(p_mj_x_N_j, axis = 1)/bn.nansum(len_arr, axis = 1)

        #STEP 3
        #caculate the new p_mj from I_m, sigma_m and P_m
        p_mj = [np.multiply(P_m, gauss(intensity_arr[0][j], I_m,sigma_m)) for j in range(j_length)]
        bottom_sum = np.reshape(np.repeat(np.reshape(np.sum(p_mj, axis = 1), [1,-1])[0],total_clusters+1), [-1,total_clusters+1])
        p_mj = np.transpose(np.divide(p_mj,bottom_sum))

        #calculate the difference
        delta_p_mj = np.sum(abs(np.subtract(p_mj, old_p_mj)))
        iter = iter +1
      ###end while

      # truncate I_m and std_m to check if the assigned states are the same to the 1's place
      I_m_trunc = [int(a) for a in I_m if a!=0 and not math.isnan(a)]
      std_trunc = np.array([int(a * 10**2)/10**2 for a in sigma_m if a!=0 and not math.isnan(a)])

      I_m_lst = []

      #assign the parameters of each state based on the condesned values and the maximum occupation probability in p_mj
      for section in range(j_length):
        arg_max = np.argmax(p_mj[:,section])
        I_m_lst.append(int(I_m[arg_max]))
        Yem[total_clusters][section]['state'] = arg_max
        Yem[total_clusters][section]['intensity'] = I_m[arg_max]
        Yem[total_clusters][section]['sigma'] = sigma_m[arg_max]
        Yem[total_clusters][section]['prob'] = np.sum(p_mj[:, arg_max])
        Yem[total_clusters][section]['pk'] = P_m[arg_max]
        Yem[total_clusters][section]['nos'] = len(np.unique(I_m_trunc))

        #Sometimes the EMOpt will say there are two states but that all segments are in the same state
        #   this if statement addresses that unique problem
        if section == j_length -1 and len(np.unique(I_m_lst)) != len(np.unique(I_m_trunc)):
          I_u, I_idx = np.unique(I_m_trunc, return_index=True)
          I_u = I_u[np.argsort(I_idx)]
          std_u = std_trunc[np.argsort(I_idx)]
          force_fitting(Yem, Yi, j_length, total_clusters, array_of_intensities[:][0:len(I_u)], array_of_lengths[:][0:len(I_u)], array_of_stddevs_sq[:][0:len(I_u)], I_u, std_u)
    else:
      # assign the AH data as the solution for these data
      for section in range(j_length):
        Yem[total_clusters][section]['state'] = Yi[total_clusters]['group'][section]
        Yem[total_clusters][section]['intensity'] = Yi[total_clusters]['intensity']
        Yem[total_clusters][section]['sigma'] = Yi[total_clusters]['std']
        Yem[total_clusters][section]['prob'] = 1e-12
        Yem[total_clusters][section]['pk'] = 1e-12
        Yem[total_clusters][section]['nos'] = total_clusters

  return Yem

In [11]:
#@title void force_fitting(dict Yem, dict Yi, int j_length, int total_clusters, ndarray intensity_arr, ndarray len_arr, ndarray stddev_arr, list I_u, list std_u)
#Find the fitting of the EMOpt based on the number of states it returned by reinitializing p_mj and performing
#   one more iteration of the calculating loop

def force_fitting(Yem, Yi, j_length, total_clusters, intensity_arr, len_arr, stddev_arr, I_u, std_u):
  p_mj = np.zeros([len(Yi[0]['group']), len(I_u)])
  for n in range(len(Yi[0]['group'])):
    pos_gauss = []
    for m in range(len(I_u)):
      pos_gauss.append(gauss(Yi[n]['intensity'], I_u[m], std_u[m]))
    max_loc = np.argmax(pos_gauss)
    p_mj[n][max_loc] = 1
  p_mj = np.transpose(p_mj)

  p_mj_x_N_j = np.multiply(p_mj, len_arr)
  I_m = np.nansum(np.multiply(intensity_arr, p_mj_x_N_j), axis = 1)/np.nansum(p_mj_x_N_j, axis = 1)
  sigma_m = np.sqrt(np.sum(np.multiply(stddev_arr, p_mj_x_N_j), axis = 1)/np.nansum(p_mj_x_N_j, axis = 1))
  P_m = np.nansum(p_mj_x_N_j, axis = 1)/np.nansum(len_arr, axis = 1)

  p_mj = [[P_m[m]*p_mj[m][j] for m in range(len(I_u))] for j in range(j_length)]
  p_mj = np.transpose(p_mj)

  for section in range(j_length):
    arg_max = np.argmax(p_mj[:,section])
    Yem[total_clusters][section]['state'] = arg_max
    Yem[total_clusters][section]['intensity'] = I_m[arg_max]
    Yem[total_clusters][section]['sigma'] = sigma_m[arg_max]
    Yem[total_clusters][section]['prob'] = np.sum(p_mj[:, arg_max])
    Yem[total_clusters][section]['pk'] = P_m[arg_max]
    Yem[total_clusters][section]['nos'] = len(np.unique(I_m))

In [12]:
#@title ndaray state_para = get_state(int Ns, int k, ndarray Yem)

def get_state(Ns, k, Yem):

  # Extract parameters of states.

  # state_para: 
  # 1st column: intensity levels of states.
  # 2nd column: noise levels(standard deviation) of states.
  # 3rd column: populations of states.
  # the number of rows in state_para corresponds to the number of states,
  # each row include the parameters from one state.
  # other parameters are added later
  # 4th column: <t> for each state
  # 5th column: time of first dark event
  
  state_para = np.zeros([Ns,6])
  g = 0
  s = 0
  while s < Ns:
      if ~np.isin(Yem[k][g]['intensity'],state_para[:,0]):
          state_para[s,0] = Yem[k][g]['intensity']
          state_para[s,1] = Yem[k][g]['sigma']
          state_para[s,2] = Yem[k][g]['pk']
          s = s + 1
      g = g + 1
      if g >= len(Yem):
          break
  
  return state_para

In [13]:
#@title ndarray state_trace = plotting_prework(int k, ndarray state_para, dict Yem, dict Yi, ndarray data)
# this guy just gets some stuff ready for the plotting we want to do later

def between(num, lower, upper):
  return (num >= lower and num <= upper)


def plotting_prework(k, state_para, Yem, Yi, data):
  #sorts the returned state_para by its order of occurence in the assignment
  temp = [Yem[k][n]['state'] for n in range(len(Yem))]
  aaa, bbb = np.unique(temp, return_index=True)
  aaa = aaa[np.argsort(bbb)]

  state_para1 = np.zeros([int(np.amax(aaa)+1),6])
  for n in range(len(aaa)):
    state_para1[aaa[n],:] = state_para[n,:]

  state_trace = np.empty([len(data),2])
  state_trace[:] = 0

  #defines the relevant ranges to account for some fast events
  ranges = []
  for n in range(len(state_para1)):
    ranges.append([state_para1[n,0]-2.5*state_para1[n,1], state_para1[n,0]+2.5*state_para1[n,1]])


  #assign each time-point to a state and intensity level
  #also accounts for short time changes which changepoint doesn't pick up
  end_ = 0
  for n in range(len(Yi)):
    start_ = end_
    end_ = start_+Yi[n]['t']
    for x in range(start_, end_):
      state_trace[x,0] = Yem[k][n]['state']

      if not between(data[x], ranges[int(state_trace[x,0])][0], ranges[int(state_trace[x,0])][1]):
        state = 0
        while state < len(state_para[:,0]) and not between(data[x], ranges[state][0], ranges[state][1]):
          state = state+1
        if state != len(state_para[:,0]):
          state_trace[x,0] = state
      state_trace[x,1] = state_para1[int(state_trace[x,0]), 0]

  #figure out time of first dark event (defined as an event not in the highest intensity state) and return  
  f = np.argmax(state_para1[:,0])
  loc = np.argwhere(state_trace[:,0]!= f)

  if len(loc) >= 1:
    for n in range(len(state_para1)):
      if len(state_para1[0,:]) >1:
        state_para1[n,5] = loc[0]*0.2
      else:
        state_para1[n,5] = 999999
  else:
    state_para1[0,5] = 999999


  return state_trace, state_para1

In [14]:
#@title list counts_x, list prob_y, list y_fit, list params = power_law_fitting(ndarray state_trace, ndarray time, int level_of_interest)
#fits segmented data to power law (a*t^(-m)) or truncated power law(a*t^(-m)*e^(t/Tc))

def calc_avg_t(b, t_min, t_max):
  #returns average time for power law
  return (b+1)/(b+2) * (t_max**(b+2) - t_min**(b+2))/(t_max**(b+1) - t_min**(b+1))

def calc_avg_t_trunc(b, t_c, t_min, t_max):
  #returns average time for truncated power law
  f = -t_c*(mpmath.gammainc(b+2, -t_max/t_c) -mpmath.gammainc(b+2, -t_min/t_c))/(mpmath.gammainc(b+1, -t_max/t_c) -mpmath.gammainc(b+1, -t_min/t_c))
  return float(mpmath.re(f))

def power_law_fitting(state_trace, time, level_of_interest):
  t = time[1]
  time_bin = time[1]
  lst = []

  #find all events associated with an intensity level and record their durration
  for n in range(len(time)-1):
    if state_trace[n,0] == level_of_interest:
      if state_trace[n,0] == state_trace[n+1,0]:
        t += time_bin
      else:
        lst.append(t)
        t = time_bin
  lst.append(t+time_bin)

  #histogram the intenisity events
  counts = np.histogram(lst, bins = time)
  counts_x = [counts[1][x] for x in range(len(counts[0])) if counts[0][x] != 0]
  counts_y = [counts[0][x] for x in range(len(counts[0])) if counts[0][x] != 0]
  counts_x.append(time[-1])

  #calcuate the probability of an event lasting a durration t
  prob_y = []
  for n in range(len(counts_y)):
    if n == 0:
      prob_y.append(counts_y[n]/((counts_x[n+1] - time_bin)/2))
    else:
      prob_y.append(counts_y[n]/((counts_x[n+1] - counts_x[n-1])/2))

  counts_x.pop(-1)

  #fit to linear power law
  power_fit, power_cov = curve_fit(linear_power_law, counts_x, np.log(prob_y))
  power_y = power_fit[0]*counts_x**(-power_fit[1])

  #fit to truncated power law
  trunc_power_fit, trunc_power_cov = curve_fit(linear_trunc_power_law, counts_x, np.log(prob_y))
  trunc_power_y = trunc_power_fit[0]*counts_x**(-trunc_power_fit[1])*np.exp(counts_x/trunc_power_fit[2])

  #determine R^2 of both fits
  R_sq_power = R_sq(linear_power_law(counts_x, power_fit[0], power_fit[1]), np.log(prob_y))
  R_sq_trunc_power = R_sq(linear_trunc_power_law(counts_x, trunc_power_fit[0], trunc_power_fit[1], trunc_power_fit[2]), np.log(prob_y))

  #return the parameters of the fit
  if R_sq_power > R_sq_trunc_power:
    y_fit= power_fit[0]*counts_x**(-power_fit[1])
    params = [power_fit[1], math.nan, R_sq_power, calc_avg_t(-power_fit[1], counts_x[0], counts_x[-1])]
  else:
    y_fit = trunc_power_fit[0]*counts_x**(-trunc_power_fit[1])*np.exp(counts_x/trunc_power_fit[2])
    params = [trunc_power_fit[1],trunc_power_fit[2],R_sq_trunc_power, calc_avg_t_trunc(-trunc_power_fit[1], trunc_power_fit[2], counts_x[0], counts_x[-1])]


  return counts_x, prob_y, y_fit, params

In [15]:
#@title BIC_calc(Yi, Yem)
#calculate the BIC for each fitting to determine the most likely fitting

@jit
def BIC_calc(Yi, Yem):
  BIC = np.zeros([len(Yi)])
  total_points = 0
  for m in range(len(Yi)):
    #for run time purpose we crop to a maximum to 10 possible states
    if m <=10:
      double_sum = 0
      L_em = 0
      G = Yem[m][0]['nos']
      total_points = 0
      num_segs = 1
      last_seg = Yem[m][0]['state']
      for j in range(len(Yi)):
        for i in range(Yi[j]['t']):
          if gauss(Yi[j]['tr'][i], Yem[m][j]['intensity'], Yem[m][j]['sigma']) == 0:
            #log(0) -> goes to -inf so we add a very large negative number
            double_sum = double_sum + math.log(Yem[m][j]['pk']) + -9999999
          else:
            double_sum = double_sum + math.log(Yem[m][j]['pk']) + math.log(gauss(Yi[j]['tr'][i], Yem[m][j]['intensity'], Yem[m][j]['sigma']))

        if math.isnan(Yem[m][j]['prob']):
          L_em = L_em + np.log(1e-99)
        else:
          L_em = L_em + np.log(Yem[m][j]['prob'])

        total_points = total_points + Yi[j]['t']

        if Yem[m][j]['state'] != last_seg:
          num_segs += 1
          last_seg = Yem[m][j]['state']

      BIC[m] = double_sum + L_em - (3/2)*G*math.log(num_segs) - num_segs*math.log(total_points)/2
    else:
      BIC[m] = -1E22  
  return BIC

In [16]:
#@title Plotting(BIC, k,  Ns_range, state_trace, data, state_para, time)
#all the plotting jazz for the calculations done here

def Plotting(BIC, k,  Ns_range, state_trace, data, state_para, time, draw_plots):

  bins_ = np.arange(np.amin(data)-10, np.amax(data)+10, 0.5)
  h = np.histogram(data, bins=bins_)

  #BIC PLOT
  if draw_plots:
    plt.subplot(2,4,1)
      #plot BIC
    plt.plot(BIC, color = my_colors[0], linewidth = 3, zorder = 0)
      #draw circle around selected max location
    plt.scatter(k, BIC[k], edgecolors = my_colors[-1], facecolors = 'none', zorder = 1, s = 100, linewidths = 3)
    plt.xlabel("grouping number")
    plt.ylabel("BIC")
    plt.ylim(np.amax([BIC[k]-2000, np.amin(BIC[0:8])-500]), np.amax(BIC)+100)
    plt.xlim([0,8])
      #plot the final number of states per grouping on right yaxis
    plt.gca().twinx().plot(Ns_range, color = my_colors[-1], linewidth = 3)
    plt.ylabel("number of states", color = my_colors[-1])
    plt.ylim([0, 10])
    plt.xticks(np.arange(0, 10, 2))
    plt.yticks(np.arange(0, 11, 2))
    plt.gca().spines['right'].set_color(my_colors[-1])
    plt.gca().tick_params(axis='y', colors=my_colors[-1])

    #POWER LAW PLOTS
    if Ns_range[k] > 1:
      for n in range(2,3+int(np.amax(state_trace[:,0]))):
          #do power law fitting
        counts_x, prob_y, y_fit, params = power_law_fitting(state_trace, time, n-2)
        state_para[n-2,3] = params[0]    #alpha
        state_para[n-2,4] = params[3]    #avg time
        plt.subplot(2,4,n)
          # plot fit power law
        plt.loglog(counts_x, y_fit, color = my_colors[0], linewidth = 2)
          # plot observed event lengths by probability
        plt.scatter(counts_x, prob_y, color = my_colors[n-1])
          # display parameters of the fitting
        plt.text(np.amin(counts_x), np.amin(prob_y)+.1, "state: " + str("{:.0f}".format(state_para[n-2,0]))+ "\n \nalpha: " + str("{:.2f}".format(params[0])) +"\nT_c: " + str("{:.2f}".format(params[1])) + "\nR^2: " + str("{:.2f}".format(params[2])) + "\navg_t (s): " + str("{:.2f}".format(params[3])))
        plt.xlabel('event time (s)')
        plt.ylabel('event probability')


    #STATE TRACE PLOT
    plt.subplot(2,4,(5,7))
      #plot data
    plt.plot(time,data, color = my_colors[0], zorder = 0)
    plt.xlim([-5,int(np.amax(time)+5)])
    for n in range(0, int(np.amax(state_trace[:,0])+1)):
      temp_trace = [state_trace[x,1]  if state_trace[x,0] == n else math.nan for x in range(len(state_trace[:,0]))]
        #plot traces of each intensity state
      plt.scatter(time, temp_trace, color = my_colors[n+1], linewidth = 3, s = 10, zorder = n+1, alpha=1)
    plt.xlabel("time (s)")
    plt.ylabel("Intensity (counts/s)")

    #INTENSITY HISTOGRAM PLOT
    plt.subplot(2,4,8)
      #histogram the data
    plt.hist(data, orientation='horizontal', bins= bins_, color = my_colors[0])
    plt.ylabel("Intensity (counts/s)")
    for n in range(len(state_para[:,0])):
      bis_bis = bisect.bisect(bins_, state_para[n,0])-1
      while h[0][bis_bis] == 0:
        bis_bis += 5
      height = h[0][bis_bis]
      gaus = norm.pdf(bins_, state_para[n,0], state_para[n,1])
        #print the gaussians of the intensity states
      plt.plot(gaus/np.amax(gaus)*height, bins_, linewidth = 5, color = my_colors[n+1])
      t = plt.text(height+2,state_para[n,0], str("{:.0f}".format(state_para[n,0])) + " +/- " + str("{:.0f}".format(state_para[n,1])))
      t.set_bbox(dict(facecolor='white', alpha=1, edgecolor='white'))
    plt.xlabel("counts")

    plt.subplots_adjust(left=0.125, bottom=0.1, right=1.1, top=0.9, wspace=0.3, hspace=0.2)
    fig = plt.gcf()
    fig.set_size_inches(20, 10)
  else:
    if Ns_range[k] > 1:
      for n in range(0,int(np.amax(state_trace[:,0]))+1):
        #do power law fitting
        counts_x, prob_y, y_fit, params = power_law_fitting(state_trace, time, n)
        state_para[n,3] = params[0]    #alpha
        state_para[n,4] = params[3]    #avg time

In [17]:
#@title CP_MAIN
# Determine the number of states and state parameters from input trajectory.
# data: raw trajectory (in the format of row).
# time: time points associated with each data point in the trajectory
# draw_plots: T/F to indicate if the data should be plotted

def CP_MAIN(data, time, draw_plots):
  #FIND THE CHANGE POINTS
  all_cp = findcp(data)
  all_cp = list(all_cp)
  if all_cp[-1] == len(data):
    all_cp.pop(-1)

  #AH CLUSTERING
  Yi = AHclusterN(data, all_cp)

  #EM OPTIMIZATION
  Yem = EMclusterN(Yi)

  #make sure the states are in contiguous order
  for n in range(len(Yem)):
    z = [Yem[n][f]['state'] for f in range(len(Yem))]
    z_rank = rankdata(z, method='dense')
    for f in range(len(Yem)):
      Yem[n][f]['state'] = z_rank[f]-1

  #CALCULATE BIC
  BIC = BIC_calc(Yi, Yem)

  #find ideal grouping
  k = np.argmax(BIC)

  Ns = Yem[k][0]['nos']
  #print(Ns)
  Ns_range = [Yem[n][0]['nos'] for n in range(len(Yem))]
  state_para = get_state(Ns,k,Yem)
  np.set_printoptions(suppress=True)

  state_trace, state_para = plotting_prework(k, state_para, Yem, Yi, data)

  Plotting(BIC, k,  Ns_range, state_trace, data, state_para, time, draw_plots)

  return state_para

In [18]:
#change file paths to select your data
parent_folder = "/content/drive/Shareddrives/Ginger Lab/IMOD/g14 data/12823_OAMOLAM_4005/analyzed_data/"   #change to master folder
qd_trace_folder = parent_folder + "particle_picking/traces/"
qd_file_name = "ROI1_50ms_8000im_1_MMStack_Default.csv"                          #change to name of trace

para_file_name = "CPA Para " + qd_file_name
fig_file_name = qd_file_name[:-4]

para_folder = parent_folder + "CPA_params/"
fig_folder = para_folder + "Example CPA Fittings/"

#read in traces
qd_traces = pd.read_csv(qd_trace_folder+ qd_file_name)
qd_traces = qd_traces.to_numpy()

int_time_s = 0.05                 ## step_size for equidistance time bins

start_trace = 1                   ## start at 1 for new batch - change for picking up in the middle of a batch

auto_save = 10                    ## save the list of params every so many traces(helps with long run times)

if os.path.exists(para_folder) == False:
  os.mkdir(para_folder)

if os.path.exists(fig_folder) == False:
  os.mkdir(fig_folder)

In [ ]:
###### UNCOMMENT FOR SYNTHETIC DATA
# data, time, act_cp = create_data(140, 100, 20, 0.05)          
# para = CP_MAIN(data, time, True)
######


###### UNCOMMENT FOR REAL DATA
lst_para = []
time = [n*int_time_s for n in range(len(qd_traces[:,0]))]

for n in range (start_trace, len(qd_traces[0])): #should start indexing at  one, slice zero is the bin numbers
  try:
    #analyze the data - every 10th trace is plotted and saved
    if n % 10 == 0:
      para = CP_MAIN(qd_traces[:,n], time, True)
      plt.savefig(fig_folder+f'QD_{n}__'+fig_file_name+'.jpg', dpi = 300, bbox_inches = 'tight')
      plt.close(plt.gcf())
    else:
      para = CP_MAIN(qd_traces[:,n], time, False)   #False - no plotting individual traces

    #append trace CPA parameters to the list of all parameters
    for j in range(len(para)):
      for k in para[j,:]:
        lst_para.append(k)
    for f in range(6):
      lst_para.append(math.nan)

    print(f"{n} -- ({len(para)})")
  except:
    print(f"failed {n}")

  #auto save in the middle of a batch
  if n%auto_save == 0:
    lst_para = np.array(lst_para).reshape([int(len(lst_para)/6), 6])
    p_name = para_file_name[:-4] + f'({n}).csv'
    pd.DataFrame(np.array(lst_para)).to_csv(os.path.join(para_folder,p_name))
    lst_para = []

#save at the end of a batch
lst_para = np.array(lst_para).reshape([int(len(lst_para)/6), 6])
p_name = para_file_name[:-4] + f'({n}).csv'
pd.DataFrame(np.array(lst_para)).to_csv(os.path.join(para_folder,p_name))
######